## Coursera Capstone Project 
### Customer Segmentation and Clustering based on Location Amenities

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course")

Hello Capstone Project Course


In [24]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=e99446f77502fa30499e481cd5a3e2f088a1c7fa64b3b41e1ce2ffcf8fda277a
  Stored in directory: c:\users\visha\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install tabulate

  Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23274 sha256=21b15d871f286de389368a19bc5d752f800fef9ff52ecab620e018a3cd9c56ee
  Stored in directory: c:\users\visha\appdata\local\pip\cache\wheels\86\8c\d9\3b6c2f55efaefe656c52b725d2fdc5669085c83444979db8b3
Successfully built tabulate
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.8.2-py3-none-any.whl (106 kB)
  Using cached soupsieve-2.0-py2.py3-none-any.whl (32 kB)



In [3]:
pip install requests

  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.25.8-py2.py3-none-any.whl (125 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)



In [65]:
pip install csv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


# Data Science Problem Definition
To identify and explore features of Manhattan and Torronto, to come up the conclusions where are the better business prospects in terms of opening restaurant. Unerstanding which different stores are available in neighbourhoods. We want to explore how much they are similar or dissimilar in aspects from a tourist point of view regarding food, accommodation and culture.

# Data Description

Using Foursquare API - explore neighbourhoods data of Toronto and Manhattan cities.
Focus on the data releven to veneus related restaurants, eateries, coffee shops and entertainment venues. Analyze this information to decide on which city is more suitable for opening the restaurant considering different food items and based on different types of visitors and tourist. 

### Intoduction
In this Project we will be webscrapping the data for Torronto and its neighbourhood. Derive Latitude and Longitudes for all the neighbourhood locations. Explore to get common venue categories in each neighbourhood, and use the features to group the neighbourhood into clusters.

We have already analysed the New York and Manhattan top 10 locations, we will be using New York data with Torronto to find top 10 locations of Torronto.

## 1. Read url and scrape required data from using BeautifulSoup library

In [19]:
# Import libraries

import pandas as pd
import numpy as np
import requests
import csv
from bs4 import BeautifulSoup

# Read source URL & Parse using lxml
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

CSV_FILE_NAME = 'cms_scrape.csv'

# Open CSV File
csv_file = open(CSV_FILE_NAME,'w',newline='')

csv_writer = csv.writer(csv_file)
# print(type(csv_writer))

# Write header column names to the file
csv_writer.writerow(['Postal Code', 'Borough', 'Neighbourhood'])

# Loop through table rows
for tr in soup.find_all('tr'):
    
## For each row traverse to each columns    
    for td in tr.find_all('td'):
        
## Read Postal Code        
        try:
            post_code = td.p.b.text                    
        except Exception as e:
            post_code = None
#         print('Postal Code :', post_code)    

## Read Borough
        try:
            borough = td.p.span.a.text
        except Exception as e:
            borough = None
#         print('Borough :', borough)
        
## Read Neighbourhoood    
        try:
            first_link = td.p.span.a            
            neighbour = []
            for a in first_link.find_next_siblings("a"):
                if a.text:
                  neighbour.append(a.text)          
        except Exception as e:
            neighbour = None
            
## Seprate neighbourhood with comma for multiple values        
        try:            
            neighbour_str = ', '.join([str(x) for x in neighbour])
        except Exception as e:
            neighbour_str = None
        
#         print('Neighbour :',neighbour_str)        

## Write the row record to file
        csv_writer.writerow([str(post_code), str(borough), str(neighbour_str)])

csv_file.close()    


## 2. Read the data from CSV File to Dataframe

In [20]:
df = pd.read_csv(CSV_FILE_NAME)
df.shape

(245, 3)

In [21]:
df = df[df['Borough']!= 'None']
df.shape

(101, 3)

In [22]:
df.head(101)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [23]:
# Setting the Neighbourhood values with Borough where Neighbourhood is Not Assigned

df.isna().sum()

Postal Code       0
Borough           0
Neighbourhood    13
dtype: int64

In [24]:
df['Neighbourhood'].fillna(df['Borough'], inplace=True)
df.head(50)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [25]:
df_new = df[df['Postal Code'].isin(['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]
df_new.reset_index(inplace=True, drop=True)
df_new

,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M1B,Scarborough,"Malvern, Rouge"
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M4G,East York,Leaside
4,M5G,Downtown Toronto,Bay Street
5,M2H,North York,Hillcrest Village
6,M1J,Scarborough,Scarborough Village
7,M9L,North York,Humber Summit
8,M1R,Scarborough,"Wexford, Maryvale"
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


## Geo Code Data for Postal Codes

In [26]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

df_geo_codes = pd.read_csv('Geospatial_Coordinates.csv')
df_geo_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Postal Codes with Latitudes and Longitudes

In [27]:
df_geo = df.join(df_geo_codes.set_index('Postal Code'),on='Postal Code')
df_geo

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [28]:
df_geo_new = df_new.join(df_geo_codes.set_index('Postal Code'),on='Postal Code')
df_geo_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M4G,East York,Leaside,43.709060,-79.363452
4,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383
5,M2H,North York,Hillcrest Village,43.803762,-79.363452
6,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
7,M9L,North York,Humber Summit,43.756303,-79.565963
8,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [29]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [31]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [39]:
df_neighborhoods = df_geo[df['Borough']=='Downtown Toronto']
df_neighborhoods

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383
41,M6G,Downtown Toronto,Downtown Toronto,43.669542,-79.422564
49,M5H,Downtown Toronto,"Richmond, King",43.650571,-79.384568
58,M5J,Downtown Toronto,"Harbourfront, Union Station, Toronto Islands",43.640816,-79.381752
67,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
76,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [40]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Torronto City are 43.6541737, -79.38081164513409.


In [47]:
print(len(df_neighborhoods))
# create map of Downtown Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# # add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)    
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

18


In [48]:
CLIENT_ID = 'BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR' # your Foursquare ID
CLIENT_SECRET = 'ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR
CLIENT_SECRET:ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR


In [49]:
df_toronto = df_neighborhoods
df_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383
41,M6G,Downtown Toronto,Downtown Toronto,43.669542,-79.422564
49,M5H,Downtown Toronto,"Richmond, King",43.650571,-79.384568
58,M5J,Downtown Toronto,"Harbourfront, Union Station, Toronto Islands",43.640816,-79.381752
67,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
76,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [50]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_geo['Borough'].unique()),
        df_geo.shape[0]
    )
)

The dataframe has 25 boroughs and 101 neighborhoods.


In [109]:
df_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383
41,M6G,Downtown Toronto,Downtown Toronto,43.669542,-79.422564
49,M5H,Downtown Toronto,"Richmond, King",43.650571,-79.384568
58,M5J,Downtown Toronto,"Harbourfront, Union Station, Toronto Islands",43.640816,-79.381752
67,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
76,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [64]:
# df_toronto.loc[4,'Neighbourhood']
# df_toronto.loc[4,'Latitude']

In [66]:
neighborhood_latitude = df_toronto.loc[4,'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[4, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [67]:
# type your answer here
LIMIT = 100
radius = 500
# search_query = 'Marble Hill'
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR&client_secret=ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [68]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e76a97c3907e70026e2a691'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [80]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [100]:
# venues = results['response']['groups'][0]['items']
venues = results['response']['groups'][0]['items']
# print("Venues : ",venues)

nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues
# print("Near by Venues : ", nearby_venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
# nearby_venues['venue.categories']
nearby_venues
# print("Near by Venues : ", nearby_venues)

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues['venue.categories'].unique()

# print("Near by Venues (Unique): ", nearby_venues)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
# nearby_venues.columns

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues

46 venues were returned by Foursquare.


<ipython-input-100-861341c459cc>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Corktown Common,Park,43.655618,-79.356211
6,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
7,The Distillery Historic District,Historic Site,43.650244,-79.359323
8,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
9,Distillery Sunday Market,Farmers Market,43.650075,-79.361832


In [101]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#     print(nearby_veunues.head())
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [111]:
# type your answer here
# print(df_toronto)
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'])
# toronto_venues

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Downtown Toronto
Bay Street
Downtown Toronto
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
Downtown Toronto
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [112]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,"Regent Park, Harbourfront",43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
7,"Regent Park, Harbourfront",43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
8,"Regent Park, Harbourfront",43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
9,"Regent Park, Harbourfront",43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,Farmers Market


In [116]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,83,83,83,83,83,83
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",16,16,16,16,16,16
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,169,169,169,169,169,169
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",100,100,100,100,100,100
"Kensington Market, Chinatown, Grange Park",87,87,87,87,87,87


In [118]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 209 uniques categories.
